In [0]:
!pip install warc
!pip install warc3-wet

In [0]:
!pip install langdetect

In [0]:
!mkdir cc_data

In [0]:
!rm -rf cc_data

In [0]:
import requests
import argparse
import time
import json
import gzip
from io import BytesIO
from bs4 import BeautifulSoup
import sys
import csv
from glob import glob
import warc
import urllib
import urllib.request
import re
from langdetect import detect

index_list = ["2019-09", "2019-13"]
keywords = 'sports' or 'basketball' or 'baseball' or 'hockey' or 'football'

def search_domain(domain):
    record_list = []
    
    for index in index_list:
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s/&filter=mime:text/html&limit=5&output=json" % domain

        response = requests.get(cc_url)
        
        if response.status_code == 200:
            records = response.content.splitlines()
            for record in records:
                record_list.append(json.loads(record))  
    return record_list     

def download_page(records):
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    fcc=open("cc_raw_final.txt",'a')
    c = 1
    link_count = 0
    for record in records:
        offset, length = int(record['offset']), int(record['length'])
        offset_end = offset + length - 1
        dummy = record['filename'].replace('/warc/','/wet/').replace('.warc.','.warc.wet.')
        link = prefix + dummy
        holder = "cc_data/"+str(c) + ".warc.wet.gz"
        try:
            if(urllib.request.urlretrieve(link,holder)):
                c = c + 1
            warc_files = glob(holder)

            for fn in warc_files:
                f = warc.open(fn)
                for record in f:
                    url = record.header.get('warc-target-uri', None)
                    if not url:
                        continue
                    #else :
                    text = record.payload.read()
                    if(detect(text.decode("utf-8")) == 'en'):
                        if  keywords in text.decode("utf-8").lower():
                            fcc.write(str(text, 'utf-8'))
                            fcc.write('\n')
                            fcc.write('\n\n\n\n\n')
                            link_count = link_count + 1
        except Exception:
            pass
    fcc.close()
    print("number of links parsed:",link_count)

if __name__ == '__main__':
    print("Starting CommonCrawl Search")

    domain = "*.wikipedia.org"
    record_list = search_domain(domain)
    demo = download_page(record_list)
    print("all done:", demo)


Starting CommonCrawl Search
number of links parsed: 110
all done: None
